In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
import matplotlib.pyplot as plt


Using TensorFlow backend.


# Preprocessing Data

In [2]:
batch_size = 128
num_classes = 10
epochs = 2



(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)


Data_Train=np.hstack([X_train,y_train])
Data_Test=np.hstack([X_test,y_test])

X_train_1=Data_Train[Data_Train[:,784]<5]
X_test_1=Data_Test[Data_Test[:,784]<5]

X_train_2=Data_Train[Data_Train[:,784]>=5]
X_test_2=Data_Test[Data_Test[:,784]>=5]

y_train_1= X_train_1[:,784]
y_train_2= X_train_2[:,784]
y_test_1 = X_test_1[:,784]
y_test_2 = X_test_2[:,784]

X_train_1= X_train_1[:,0:784]
X_train_2= X_train_2[:,0:784]
X_test_1= X_test_1[:,0:784]
X_test_2= X_test_2[:,0:784]


y_train_1 = keras.utils.to_categorical(y_train_1, num_classes)
y_test_1 = keras.utils.to_categorical(y_test_1, num_classes)
y_train_2 = keras.utils.to_categorical(y_train_2, 10)
y_test_2 = keras.utils.to_categorical(y_test_2, 10)
y_test = keras.utils.to_categorical(y_test, 10)

X_train_1= X_train_1.reshape(len(X_train_1),28,28,1)
X_train_2= X_train_2.reshape(len(X_train_2),28,28,1)
X_test_1= X_test_1[:,0:784].reshape(len(X_test_1),28,28,1)
X_test_2= X_test_2[:,0:784].reshape(len(X_test_2),28,28,1)
X_test = X_test.reshape(len(X_test),28,28,1)

# Compiling the CNN model to train on numbers 0 to 4

In [7]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', name='conv_1', 
                 input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2), name='maxpool_1'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='conv_2'))
model.add(MaxPooling2D((2, 2), name='maxpool_2'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', name='dense_1'))
#model.add(Dense(128, activation='relu', name='dense_2'))
model.add(Dense(num_classes, activation='softmax', name='output'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 14, 14, 16)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 14, 14, 32)        4640      
_________________________________________________________________
maxpool_2 (MaxPooling2D)     (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               401664    
__________

# Fitting model on training set 1 and testing on test set

In [8]:
history = model.fit(X_train_1, y_train_1,
                    batch_size=batch_size,
                    epochs=2,
                    verbose=1,
                    validation_data=(X_test_1, y_test_1))
                    
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 30596 samples, validate on 5139 samples
Epoch 1/2
30596/30596 [==============================] - 64s 2ms/step - loss: 0.0304 - categorical_accuracy: 0.9454 - val_loss: 0.0053 - val_categorical_accuracy: 0.9926
Epoch 2/2
10000/10000 [==============================] - 9s 885us/step
Test loss: 0.8512219931602478
Test accuracy: 0.5107


# Freezing all the layers of CNN model prior to softmax and adding one more layer to the model  to train on numbers 5 to 9

In [9]:
for layer in model.layers:
    layer.trainable=False

model.pop()
model.add(Dense(128, activation='relu', name='dense_2'))
model.add(Dense(num_classes, activation='softmax', name='output'))
    
    
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 14, 14, 16)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 14, 14, 32)        4640      
_________________________________________________________________
maxpool_2 (MaxPooling2D)     (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               401664    
__________

# Fitting model on training set 2 and testing on test set

In [10]:
history = model.fit(X_train_2, y_train_2,
                    batch_size=batch_size,
                    epochs=2,
                    verbose=1,
                    validation_data=(X_test_2, y_test_2))
                    
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 29404 samples, validate on 4861 samples
Epoch 1/2
29404/29404 [==============================] - 32s 1ms/step - loss: 0.0760 - categorical_accuracy: 0.8566 - val_loss: 0.0315 - val_categorical_accuracy: 0.9502
Epoch 2/2
29404/29404 [==============================] - 30s 1ms/step - loss: 0.0414 - categorical_accuracy: 0.9251 - val_loss: 0.0226 - val_categorical_accuracy: 0.9644
Test loss: 0.8163049454689026
Test accuracy: 0.4688
